In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('Training_set.csv')
imgPath = data['filename']
paths = r'C:\Users\cw\Downloads\weather'+'\\'+ imgPath

df = pd.DataFrame( columns=['Image_Path', 'Labels'])
df['Image_Path'] = paths
df['Labels'] = data['label']

labels = np.unique(data['label'])

In [10]:
im_size = 32
train_image = []
count = 0
for path,label in zip(paths,data.label):
        img = cv2.imread(path)
#         %matplotlib inline
        #train_image.append(cv2.resize(img,(im_size,im_size)))
#         plt.imshow(img)
#         plt.title(label)
#         plt.show()
   

In [7]:
train_image = np.array(train_image)
train_image = train_image/255.0

In [8]:
y = data['label'].values

y = pd.get_dummies(y)
y = np.array(y)
y

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [9]:
import tensorflow as tf


In [11]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (im_size,im_size,3)),
                            tf.keras.layers.MaxPool2D(2,2),
                            #
                            tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            #
                            tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            ##
                            tf.keras.layers.Flatten(),
                            #
                            tf.keras.layers.Dense(512, activation='relu'),
                            #
                            tf.keras.layers.Dense(5,activation='softmax')
                           
                           ])

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(0.001),
              metrics = ['accuracy']
             )

In [ ]:
model.fit(train_image, y,
         epochs=30,
         batch_size = 10
         )

In [ ]:
data_test = pd.read_csv('Testing_set.csv')
imgPath_test = data_test['filename']
paths_test = r'C:\Users\manish kumar\Desktop\M.tech\Bootcamp\DL\test'+'\\'+ imgPath_test
paths_test.shape

(450,)

In [ ]:
im_size = 35
test_image = []
for path in paths_test:
        img = cv2.imread(path)
#         %matplotlib inline
        test_image.append(cv2.resize(img,(im_size,im_size)))
#         plt.imshow(img)
#         plt.title(label)
#         plt.show()
   

In [ ]:
test_image = np.array(test_image)
test_image = test_image/255.0
test_image.shape

In [ ]:
prediction = model.predict(test_image)
len(prediction)

In [ ]:
label_predicted = []
for i in range(len(prediction)):
    ind = np.argmax(prediction[i])
    label_predicted.append(labels[ind])


In [ ]:
len(label_predicted)

450

In [ ]:
result = pd.DataFrame(label_predicted)
result.index = data_test['filename'].index
result.columns = ['label']
result.to_csv('prediction_result_dl4.csv',index=False)

In [ ]:
INPUT_SHAPE = (224,224, 3)

# get the VGG19 model
vgg_layers = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False,
input_shape=INPUT_SHAPE)

vgg_layers.summary()


In [ ]:
# Fine-tune all the layers
for layer in vgg_layers.layers:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in vgg_layers.layers:
    print(layer, layer.trainable)

In [ ]:
# define sequential model
model = tf.keras.models.Sequential()
# Add the vgg convolutional base model
model.add(vgg_layers)

# add flatten layer
model.add(tf.keras.layers.Flatten())

# add dense layers with some dropout
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.3))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.3))

# add output layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
loss='categorical_crossentropy',
metrics=['accuracy'])

# view model layers
model.summary()

In [ ]:
EPOCHS = 3
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,
restore_best_weights=True,
verbose=1)

history = model.fit(train_image, y,
batch_size=100,
callbacks=[es_callback],
validation_split=0.1, epochs=EPOCHS,
verbose=1)